In [1]:
import os
import yaml
from pathlib import Path
from tqdm import tqdm
from PIL import Image  # Sử dụng Pillow để lấy kích thước ảnh

In [2]:
# Giả định bạn đã cài đặt mlstruct-fp
from MLStructFP.db import DbLoader

In [6]:
# =================================================================================
# --- 1. PHẦN CẤU HÌNH (VUI LÒNG CHỈNH SỬA) ---
# =================================================================================

# Đường dẫn tới file json của database, ví dụ: 'data/fp.json'
# Sử dụng đường dẫn tương đối từ vị trí chạy script
DB_JSON_PATH = 'test/data/fp.json'

# Thư mục để lưu các file label .txt đã được chuyển đổi
OUTPUT_LABEL_DIR = 'out/yolo_format/labels/'

# File cấu hình cho YOLO
YAML_FILENAME = 'damage_dataset.yaml'

# Ánh xạ tên lớp sang chỉ số (integer). BẮT BUỘC phải bắt đầu từ 0.
# File test không cung cấp thông tin về label, vì vậy chúng ta cần giả định.
# Giả sử tất cả các Rect là một loại 'damage'.
CLASS_MAP = {
    'damage': 0,
    # Thêm các lớp khác nếu có logic để phân biệt chúng
}

In [4]:
# =================================================================================
# --- 2. SCRIPT CHUYỂN ĐỔI (DỰA TRÊN PHÂN TÍCH FILE TEST) ---
# =================================================================================

def get_image_size(image_path):
    """Sử dụng Pillow để đọc kích thước ảnh một cách an toàn."""
    try:
        with Image.open(image_path) as img:
            return img.size  # Trả về (width, height)
    except FileNotFoundError:
        print(f"⚠️ Cảnh báo: Không tìm thấy file ảnh: {image_path}")
        return None, None
    except Exception as e:
        print(f"⚠️ Lỗi khi đọc ảnh {image_path}: {e}")
        return None, None

In [7]:
"""
Sử dụng phương pháp đúng được suy ra từ file test để chuyển đổi dữ liệu.
"""
print("🚀 Bắt đầu quá trình chuyển đổi theo phương pháp đã xác thực...")

output_path = Path(OUTPUT_LABEL_DIR)
output_path.mkdir(parents=True, exist_ok=True)

# Khởi tạo DbLoader với đường dẫn tới file JSON
db = DbLoader(DB_JSON_PATH)
db_base_path = Path(db.path)

# Lấy danh sách các Floor từ loader - ĐÂY LÀ CÁCH CHÍNH XÁC
floors = db.floors

print(f"🔎 Tìm thấy {len(floors)} ảnh/bản vẽ (Floors) để xử lý.")

🚀 Bắt đầu quá trình chuyển đổi theo phương pháp đã xác thực...
🔎 Tìm thấy 8 ảnh/bản vẽ (Floors) để xử lý.


In [11]:
for floor in floors:
    # 1. Lấy đường dẫn và kích thước ảnh
    image_relative_path = floor.image_path
    if not image_relative_path:
        continue
    
    # Đường dẫn ảnh trong file json có thể là tương đối so với vị trí file json
    image_full_path = db_base_path / image_relative_path
    img_width, img_height = get_image_size(image_full_path)
    print(f"image path: {image_relative_path}")
    print(f"floor id: {floor.id}")
    print(f"width: {img_width}, height: {img_height}")
    
    if not img_width or not img_height:
        continue

    # 2. Tạo file label
    label_filename = image_full_path.stem + '.txt'
    label_filepath = output_path / label_filename
    
    # 3. Lặp qua các annotation (Rects) và chuyển đổi
    with open(label_filepath, 'w') as f_out:
        # Dựa vào file test, `floor.rect` chứa các đối tượng hình học
        for rect_obj in floor.rect:
            # Lấy polygon từ thuộc tính .points
            polygon_points = rect_obj.points
            print(polygon_points)
            if not polygon_points or len(polygon_points) < 3:
                continue
            
            # TODO: Đây là điểm bạn cần tùy chỉnh.
            # Logic để xác định class_index cho mỗi rect_obj.
            # Ở đây, chúng ta tạm gán tất cả là lớp 'damage'.
            class_index = CLASS_MAP['damage']
            
            # Chuẩn hóa tọa độ
            normalized_points = []
            for point in polygon_points:
                # `point` có thể là một object có thuộc tính .x, .y
                x_norm = point.x / img_width
                y_norm = point.y / img_height
                normalized_points.extend([x_norm, y_norm])
            print(normalized_points)
            
            points_str = " ".join(map(str, normalized_points))
            f_out.write(f"{class_index} {points_str}\n")

print(f"\n✅ Hoàn tất chuyển đổi annotation.")

# 4. Tạo file cấu hình .yaml
# create_yaml_file(output_path.parent)

image path: /mnt/c/Users/admin/source/repos/agent-proj/MLSTRUCT/MLSTRUCT-FP/test/data/2cb0cff60c42d1df4f4607b2ccb7570b0464f29e.png
floor id: 1058
width: 9600, height: 6784
[(73.868,-6.223), (92.248,-6.223), (92.248,-6.423), (73.868,-6.423)]
[0.007694583333333333, -0.0009173054245283018, 0.009609166666666667, -0.0009173054245283018, 0.009609166666666667, -0.0009467865566037736, 0.007694583333333333, -0.0009467865566037736]
[(45.068,-11.423), (92.248,-11.423), (92.248,-11.623), (45.068,-11.623)]
[0.004694583333333333, -0.001683814858490566, 0.009609166666666667, -0.001683814858490566, 0.009609166666666667, -0.0017132959905660377, 0.004694583333333333, -0.0017132959905660377]
[(18.178,-4.854), (27.778,-4.854), (27.778,-5.054), (18.178,-5.054)]
[0.0018935416666666669, -0.0007155070754716981, 0.0028935416666666667, -0.0007155070754716981, 0.0028935416666666667, -0.0007449882075471698, 0.0018935416666666669, -0.0007449882075471698]
[(45.068,-6.423), (45.068,-2.004), (45.268,-2.004), (45.268,

In [ ]:
# def create_yaml_file(dataset_root_path):
#     """Tạo file data.yaml cần thiết cho việc huấn luyện YOLO."""
#     names = sorted(CLASS_MAP, key=CLASS_MAP.get)
#     data_yaml = {
#         'path': str(dataset_root_path.resolve()),
#         'train': 'images/train', # Bạn cần tự tạo và phân chia thư mục này
#         'val': 'images/val',     # Bạn cần tự tạo và phân chia thư mục này
#         'names': names
#     }
#     yaml_path = dataset_root_path / YAML_FILENAME
#     with open(yaml_path, 'w') as f:
#         yaml.dump(data_yaml, f, sort_keys=False, default_flow_style=False)
#     print(f"👍 Đã tạo file cấu hình: {yaml_path.resolve()}")
#     print("-" * 30)
#     print("🎉 Mọi thứ đã sẵn sàng!")

# if __name__ == '__main__':
#     # Cài đặt các thư viện cần thiết:
#     # pip install pyyaml tqdm Pillow
#     convert_to_yolo_segmentation()